In [10]:
#function to test models 
import nest_asyncio
import warnings
from mb_utils.src.logging import logger

nest_asyncio.apply()
warnings.filterwarnings("ignore")

In [11]:
from mb_pytorch.utils.version import version
version

'1.0.41'

In [12]:
from mb_pytorch.dataloader.loader import DataLoader

k = DataLoader('../scripts/detection/object_detection.yaml',logger=logger)
k_data = k.data_dict['data']
model_data_dict = k.data_dict['model']

Fri 16:29:04,730 INF Read data from yaml file: [{'data': {'from_datasets': 'CelebA', 'datasets_params_train': {'root': '/home/malav/D
Fri 16:29:04,732 INF Data folder already exists. Using existing data folder :  /home/malav/Desktop/mb_packages/mb_pytorch/data/      


In [13]:
from mb_pytorch.training.train_params import train_helper
model_yaml_data = k.data_dict['model']
loss_attr,optimizer_attr,optimizer_dict,scheduler_attr,scheduler_dict = train_helper(model_yaml_data) 

In [16]:
from torch.utils.tensorboard import SummaryWriter
import os

path_logs = os.path.join(k_data['datasets_params_train']['root'], 'logs')
writer = SummaryWriter(log_dir=path_logs)


In [19]:
train_loader,val_loader,_,_ = k.data_load()


Sat 02:44:06,051 INF Data file: CelebA loading from torchvision.datasets.                                                            


Files already downloaded and verified
Files already downloaded and verified


In [30]:
targets[0][:]

tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
         1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1]])

In [32]:
for batch_idx, (images, targets) in enumerate(train_loader):
    print("image is : {}".format(images.shape))
    print("target is : {}".format(targets))
    d = {}
    d['boxes'] = targets[1][:]
    d['labels'] = targets[0][:]
    target_new = [d]
    targets = [{k: v.to('cpu') for k, v in t.items()} for t in target_new]
    print("target_new is : {}".format(targets))
    break

Sat 02:52:55,124 INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,135 INF Sat 02:52:55,135 transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': FSat 02:52:55,136 INF 
transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,143 Sat 02:52:55,145 INF INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': Ftransforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F

Sat 02:52:55,148 INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,150 INF transforms: {'transform': True, 'resize':

image is : torch.Size([4, 3, 218, 178])
target is : [tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
         1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1]]), tensor([[ 95,  71, 226, 313],
        [ 72,  94, 221, 306],
        [216,  59,  91, 126],
        [622, 257, 564, 781]])]
target_new is : [{'boxes': tensor([[ 95,  71, 226, 313],
        [ 72,  94, 221, 306],
        [216,  59,  91, 126],
        [622, 257, 564, 781]]), 'labels': tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
         1, 0, 0, 1, 0, 0, 

Sat 02:52:55,169 

INF Sat 02:52:55,170 transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': FINF 
Sat 02:52:55,171 transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,175 INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,179 INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,183 INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,190 INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Sat 02:52:55,201 INF transforms: {'transform': True, 'resize': {'val': False, '

In [17]:
from mb_pytorch.detection.training import detection_train_loop
detection_train_loop?

Signature:
detection_train_loop(
    k_yaml: dict,
    scheduler: Optional[object] = None,
    writer: Optional[object] = None,
    logger: Optional[object] = None,
    gradcam: Optional[object] = None,
    gradcam_rgb: str = False,
    device: str = 'cpu',
)
Docstring:
Function to train the model
Args:
    k_yaml: data dictionary YAML of DataLoader
    scheduler: scheduler
    writer: tensorboard writer
    logger: logger
    gradcam: gradcam layers to be visulized
    device: default is cpu
output:
    None
File:      ~/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/mb_pytorch/detection/training.py
Type:      function

In [18]:
detection_train_loop(k,model_data_dict,writer=writer)

Fri 16:29:40,991 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   
Fri 16:29:40,993 INF Data file: CelebA loading from torchvision.datasets.                                                            


Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/10 [00:00<?, ?it/s]Fri 16:29:58,424 Fri 16:29:58,425 INF INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': Ftransforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': FFri 16:29:58,426 

INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Fri 16:29:58,436 INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F
Fri 16:29:58,439 Fri 16:29:58,439 INF INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': Ftransforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': F

Fri 16:29:58,446 Fri 16:29:58,446 INF INF transforms: {'transform': True, 'resize': {'val': False, 'args': {'size': [256, 256]}}, 'random_crop': {'val': Ftran

AttributeError: 'Tensor' object has no attribute 'items'